# Data Engineering with PostgreSQL and Python3

## Psycopg 

* It is the most popular PostgreSQL database adapter for the Python programming language. 
* Its main features are the complete implementation of the Python DB API 2.0 specification and the thread safety (several threads can share the same connection). 
* It was designed for heavily multi-threaded applications that create and destroy lots of cursors and make a large number of concurrent “INSERT”s or “UPDATE”s.

### Importing useful Libraries

In [1]:
import psycopg2 as pg
import pandas as pd

### Creating Database and Connection

In [2]:
def create_database_and_connection():
    # Connecting to the default database
    try:
        conn = pg.connect(
            "host = localhost \
            dbname = test \
            user = postgres \
            password = Passdb"
    )
    except pg.Error as e:
        print(f"\n{e}\nERROR: Could not connect to the Postgress Database\n")
        
    conn.set_session(autocommit=True)
    
    # Creating initial cursor
    try:
        cur = conn.cursor()
    except pg.Error as e:
        print(f"\n{e}\nERROR: Could not make a cursor to the Postgress Database\n")
    
    # Drop if exists and create the Database
    cur.execute("DROP DATABASE IF EXISTS account")
    cur.execute("CREATE DATABASE account")
    
    # Closing connection to the default Database
    try:
        conn.close()
    except pg.Error as e:
        print(f"\n{e}\nERROR: Cannot closed the connection to the former Database")

    # Initiating Connection to the created Database with a New Role
    try:
        conn = pg.connect("host=127.0.0.1 dbname=account user=computech password=Pass")
    except pg.Error as e:
        print(f"\n{e}\nERROR: Could not connect to the Database\n")

    # Initiating a Cursor to the New Database
    try:
        cur = conn.cursor()
    except pg.Error as e:
        print(f"\n{e}\nERROR: Could not make a cursor to the Database\n")

    # Creating automatic commit
    conn.set_session(autocommit=True)
    
    return cur, conn
    
    
cur, conn = create_database_and_connection()
    

### Drop Table Function Operation

In [3]:
def drop_table(cur, conn):
    for query in dt_query:
        cur.execute(query)
        conn.commit()

### Create Table Function Operation

In [4]:
def create_table(cur, conn):
    for query in ct_query:
        cur.execute(query)
        conn.commit() 

### Importing the Dataset

In [5]:
child_malnutrition = pd.read_csv("Childhood malnutrition.csv")

In [6]:
display(child_malnutrition.head()), display(child_malnutrition.tail())

,Entity,Code,Year,"Prevalence of stunting, height for age (% of children under 5)","Prevalence of underweight, weight for age (% of children under 5)","Prevalence of wasting, weight for height (% of children under 5)"
0,Afghanistan,AFG,1997,53.2,44.9,18.2
1,Afghanistan,AFG,2004,59.3,32.9,8.6
2,Afghanistan,AFG,2013,40.4,24.6,9.5
3,Afghanistan,AFG,2018,38.2,19.1,5.1
4,Albania,ALB,1997,20.4,7.1,8.1


,Entity,Code,Year,"Prevalence of stunting, height for age (% of children under 5)","Prevalence of underweight, weight for age (% of children under 5)","Prevalence of wasting, weight for height (% of children under 5)"
912,Zimbabwe,ZWE,2009,35.1,11.7,2.4
913,Zimbabwe,ZWE,2010,32.2,10.2,3.2
914,Zimbabwe,ZWE,2014,27.6,11.2,3.2
915,Zimbabwe,ZWE,2015,27.1,8.5,3.3
916,Zimbabwe,ZWE,2019,23.5,9.7,2.9


(None, None)

### Renaming the Pandas DataFrame Columns of our Dataset

In [7]:
child_malnutrition.columns = ['Entity', 'Code', 'Year', 'stunting_height_for_age', 'underweight_weight_for_age', 'wasting_weight_for_height']

In [8]:
child_malnutrition.columns

Index(['Entity', 'Code', 'Year', 'stunting_height_for_age',
       'underweight_weight_for_age', 'wasting_weight_for_height'],
      dtype='object')

### Creating Data Structure for the Database

In [9]:
child_mal_tbl = (
    """CREATE TABLE IF NOT EXISTS child_malnutrition(
        Entity VARCHAR (150) NOT NULL,
        Code VARCHAR (3) NOT NULL,
        Year INT NOT NULL,
        stunting_height_for_age DECIMAL (10) NOT NULL,
        underweight_weight_for_age DECIMAL (10) NOT NULL,
        wasting_weight_for_height DECIMAL (10) NOT NULL
    )"""
)

### Passing the Data Structure as parameter to the Cursor's execute function

In [10]:
cur.execute(child_mal_tbl)
conn.commit()

### Inserting Data from the DataFrame into the SQL Table

In [11]:
child_mal_val = (
    """INSERT INTO child_malnutrition(
    Entity,
    Code,
    Year,
    stunting_height_for_age,
    underweight_weight_for_age,
    wasting_weight_for_height) 
    VALUES(%s, %s, %s, %s, %s, %s)
    """
)

### Passing the values as parameter to the `execute` function

In [13]:
for i, row in child_malnutrition.iterrows():
    cur.execute(child_mal_val, list(row))
conn.commit()

### Closing the Cursor and Terminating the Connection to the Database

In [14]:
try:
    cur.close()
    conn.close()
except pg.Error as e:
    print(f"\n{e}\nERROR: Could not close cursor or the connection to the Database\n")